<a href="https://colab.research.google.com/github/pocketmax/skull-ship/blob/main/autokeras_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://autokeras.com/tutorial/text_classification/



In [ ]:
!pip install --upgrade pip
!pip install autokeras
!nvidia-smi
!pip install tensorflow[and-cuda]

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files
import autokeras as ak

Using TensorFlow backend


In [ ]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=["pos", "neg"]
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=["pos", "neg"]
)

In [ ]:
# train_data.data is a list
skip=6000
x_train = np.array(train_data.data[:skip])
y_train = np.array(train_data.target[:skip])
x_test = np.array(test_data.data[:skip])
y_test = np.array(test_data.target[:skip])
# test = np.array(test_data.data[:6000])
# y_test = np.array(test_data.target) TODO: why was it like this?



In [ ]:
# TODO: What does max_trials mean?
clf = ak.TextClassifier(
    overwrite=True,
    max_trials=1
)
clf.fit(x=x_train, y=y_train, epochs=20)


In [ ]:
print(clf.evaluate(x=x_test, y=y_test))


188/188 [==============================] - 1s 4ms/step - loss: 0.6809 - accuracy: 0.8868
[0.6809044480323792, 0.8868333101272583]


In [ ]:
clf.predict(np.array([
    "that movie sucked. it didn't have enough special effects", #0
    "that movie was the best. I loved the acting", #1
    "it wasn't that good. i hated it totally. I liked some parts but I didn't like the rest", #0
    "it was a crappy movie by far. it was terrible" #0
]))


1/1 [==============================] - 0s 7ms/step


array([[0.],
       [1.],
       [0.],
       [0.]], dtype=float32)